In [1]:
import pandas as pd

nodes = pd.read_csv("jedit-comm_louvain1-nodes.csv")
# edges = pd.read_csv("jedit-comm_louvain-edges.csv")

nodes.shape

(20, 14)

note: communities [hidef] are from cytoscape's community detection with default parameters:

- algorithm = HiDeF
- weight column = none
- parameters:
  - max resolution = 50
  - consensus threshold = 75
  - persistent threshold = 5
  - algorithm = louvain
  - target community number = (blank)

communities [louvain] uses the parameters:

- algorithm = Louvain
- weight column = none
- parameters:
  - directed = True
  - overlapping = False
  - random gen seed = 42
  - resolution = 5
  - deep hierarchy = False

In [2]:
nodes['level'] = [l if l==0 else 1 for l in nodes['level']]

note: level is currently manually edited in cytoscape. should be automated:

- visited nodes <- []
- remaining nodes <- all nodes
- find sink (node with no outgoing edges)
- assign level 0 to sink
- add sink to visited nodes
- remove sink from remaining nodes
- while there are remaining nodes:
  - find nodes in remaining nodes that has edge to visited nodes
  - assign level +1 to those nodes
  - add those nodes to visited nodes
  - remove those nodes from remaining nodes

problem: cytoscape community detection export does not show edges' source and target???

In [3]:
nodes['members'] = [set(l.split(" ")) for l in nodes['CD_MemberList']]

nodes.sort_values(by=['level'], ascending=False, inplace=True)
indices = list(nodes.index)

df = nodes[['name', 'CD_MemberList_Size', 'members', 'level']]
pd.concat([df.head(5), df.tail(5)])

,name,CD_MemberList_Size,members,level
10,C17241,21,"{DockableWindowManagerImpl.Entry, DockableWind...",1.0
1,C17232,188,"{ManagePanel.KeyboardAction, PluginResURLConne...",1.0
18,C17249,2,"{JCheckBoxList.DummyRenderer, JCheckBoxList}",1.0
17,C17248,2,"{JCheckBoxList.Entry, CheckBoxListModel}",1.0
16,C17247,3,"{ExtendedGridLayout, ExtendedGridLayoutConstra...",1.0
5,C17236,119,"{BshIterator, CollectionManager.BasicBshIterat...",1.0
4,C17235,135,"{TaskMonitor.TaskTableEditor, MirrorList, VFSD...",1.0
3,C17234,170,"{PropertiesChanged, EditBus.HandlerList, View....",1.0
2,C17233,177,{ErrorsWidgetFactory.ErrorDialog.MyActionListe...,1.0
0,C17251,1046,"{MirrorList, PluginManager, DisplayManager, Ed...",0.0


In [4]:
nodes['my_members_str'] = ["" for i in indices]
df = nodes[['name', 'CD_MemberList_Size', 'members', 'my_members_str']]
pd.concat([df.head(5), df.tail(5)])

,name,CD_MemberList_Size,members,my_members_str
10,C17241,21,"{DockableWindowManagerImpl.Entry, DockableWind...",
1,C17232,188,"{ManagePanel.KeyboardAction, PluginResURLConne...",
18,C17249,2,"{JCheckBoxList.DummyRenderer, JCheckBoxList}",
17,C17248,2,"{JCheckBoxList.Entry, CheckBoxListModel}",
16,C17247,3,"{ExtendedGridLayout, ExtendedGridLayoutConstra...",
5,C17236,119,"{BshIterator, CollectionManager.BasicBshIterat...",
4,C17235,135,"{TaskMonitor.TaskTableEditor, MirrorList, VFSD...",
3,C17234,170,"{PropertiesChanged, EditBus.HandlerList, View....",
2,C17233,177,{ErrorsWidgetFactory.ErrorDialog.MyActionListe...,
0,C17251,1046,"{MirrorList, PluginManager, DisplayManager, Ed...",


In [5]:
visited = { n for n in nodes.loc[indices[0],'members'] }
nodes.loc[indices[0],'my_members_str'] = ' '.join(visited)


In [6]:
for idx in range(1,len(indices)):
  i = indices[idx-1]
  j = indices[idx]
  my_nodes = nodes.loc[j,'members'] - visited
  visited |= my_nodes
  # print(nodes.loc[j,'name'])
  # print("then",nodes.loc[j,'members'])
  # print(" now",my_nodes)
  nodes.loc[j,'my_members_str'] = ' '.join(my_nodes)

nodes['my_members'] = [set(l.split(" ")) if l else set() for l in nodes['my_members_str']]
nodes['my_members_size'] = [len(l) for l in nodes['my_members']]


In [7]:
df = nodes[['name', 'CD_MemberList_Size', 'members', 'my_members_size', 'my_members', 'level']]

pd.concat([df.head(5), df.tail(5)])

,name,CD_MemberList_Size,members,my_members_size,my_members,level
10,C17241,21,"{DockableWindowManagerImpl.Entry, DockableWind...",21,"{DockableWindowManagerImpl.Entry, DockableWind...",1.0
1,C17232,188,"{ManagePanel.KeyboardAction, PluginResURLConne...",188,"{ManagePanel.KeyboardAction, PluginManager.Act...",1.0
18,C17249,2,"{JCheckBoxList.DummyRenderer, JCheckBoxList}",2,"{JCheckBoxList.DummyRenderer, JCheckBoxList}",1.0
17,C17248,2,"{JCheckBoxList.Entry, CheckBoxListModel}",2,"{JCheckBoxList.Entry, CheckBoxListModel}",1.0
16,C17247,3,"{ExtendedGridLayout, ExtendedGridLayoutConstra...",3,"{ExtendedGridLayout, ExtendedGridLayoutConstra...",1.0
5,C17236,119,"{BshIterator, CollectionManager.BasicBshIterat...",119,"{BshIterator, CollectionManager.BasicBshIterat...",1.0
4,C17235,135,"{TaskMonitor.TaskTableEditor, MirrorList, VFSD...",135,"{TaskMonitor.TaskTableEditor, MirrorList, VFSD...",1.0
3,C17234,170,"{PropertiesChanged, EditBus.HandlerList, View....",170,"{PropertiesChanged, EditBus.HandlerList, View....",1.0
2,C17233,177,{ErrorsWidgetFactory.ErrorDialog.MyActionListe...,174,{ErrorsWidgetFactory.ErrorDialog.MyActionListe...,1.0
0,C17251,1046,"{MirrorList, PluginManager, DisplayManager, Ed...",0,{},0.0


In [8]:
from functools import reduce

all = reduce(lambda x,y: x|y, nodes['my_members'])
len(all)

1043

In [9]:
nodes.columns

Index(['CD_AnnotatedMembers', 'CD_AnnotatedMembers_Overlap',
       'CD_AnnotatedMembers_Pvalue', 'CD_AnnotatedMembers_Size',
       'CD_CommunityName', 'CD_Labeled', 'CD_MemberList',
       'CD_MemberList_LogSize', 'CD_MemberList_Size', 'level', 'name',
       'selected', 'shared name', 'wordcloud.sync', 'members',
       'my_members_str', 'my_members', 'my_members_size'],
      dtype='object')

In [10]:
c_nodes = nodes.drop(nodes[nodes.my_members_str.map(len)==0].index)
c_nodes = c_nodes.drop(columns=[
      'CD_AnnotatedMembers', 'CD_AnnotatedMembers_Overlap',
      'CD_AnnotatedMembers_Pvalue', 'CD_AnnotatedMembers_Size',
      'CD_CommunityName', 'CD_Labeled', 'CD_MemberList',
      'CD_MemberList_LogSize', 'CD_MemberList_Size',
      'selected', 'shared name', 'members', 'my_members_str', 
      'my_members_size'])
c_nodes.rename(columns={'name':'id', 'my_members':'members'}, inplace=True)
df = c_nodes
pd.concat([df.head(5), df.tail(5)])

,level,id,wordcloud.sync,members
10,1.0,C17241,False,"{DockableWindowManagerImpl.Entry, DockableWind..."
1,1.0,C17232,NaN,"{ManagePanel.KeyboardAction, PluginManager.Act..."
18,1.0,C17249,False,"{JCheckBoxList.DummyRenderer, JCheckBoxList}"
17,1.0,C17248,NaN,"{JCheckBoxList.Entry, CheckBoxListModel}"
16,1.0,C17247,NaN,"{ExtendedGridLayout, ExtendedGridLayoutConstra..."
6,1.0,C17237,NaN,"{BufferOptions, PluginManagerOptionPane.Mirror..."
5,1.0,C17236,NaN,"{BshIterator, CollectionManager.BasicBshIterat..."
4,1.0,C17235,NaN,"{TaskMonitor.TaskTableEditor, MirrorList, VFSD..."
3,1.0,C17234,NaN,"{PropertiesChanged, EditBus.HandlerList, View...."
2,1.0,C17233,False,{ErrorsWidgetFactory.ErrorDialog.MyActionListe...


In [11]:
import re
from collections import Counter

names = ['-'.join([name for name, _ in Counter(re.findall(r'[A-Z](?:[a-z]+|[A-Z]*(?=[A-Z]|$))', ' '.join(members))).most_common(3)]) for members in c_nodes['members']]

note: community nodes were named individually by examining its members. some ideas for automation:

- use the name of the largest class
- use the name of "central" class in the community (e.g., most ingoing + outgoing edges?)
- find most recurring word from class names?

In [12]:
# with open("comm_louvain.txt", "r") as file:
# 	names = [line.split(':')[0] for line in file.readlines()]
# len(names)==len(set(names))

In [13]:
c_nodes['name'] = names
c_nodes['id'] = names
c_nodes['members_str'] = [' '.join(m) for m in c_nodes['members']]
# for line in [' '.join(s) for s in c_nodes['members']]:
#   print(line)
df = c_nodes

pd.concat([df.head(5), df.tail(5)])

,level,id,wordcloud.sync,members,name,members_str
10,1.0,Window-Dockable-Handler,False,"{DockableWindowManagerImpl.Entry, DockableWind...",Window-Dockable-Handler,DockableWindowManagerImpl.Entry DockableWindow...
1,1.0,Panel-Handler-Plugin,NaN,"{ManagePanel.KeyboardAction, PluginManager.Act...",Panel-Handler-Plugin,ManagePanel.KeyboardAction PluginManager.Actio...
18,1.0,J-Check-Box,False,"{JCheckBoxList.DummyRenderer, JCheckBoxList}",J-Check-Box,JCheckBoxList.DummyRenderer JCheckBoxList
17,1.0,Check-Box-List,NaN,"{JCheckBoxList.Entry, CheckBoxListModel}",Check-Box-List,JCheckBoxList.Entry CheckBoxListModel
16,1.0,Layout-Extended-Grid,NaN,"{ExtendedGridLayout, ExtendedGridLayoutConstra...",Layout-Extended-Grid,ExtendedGridLayout ExtendedGridLayoutConstrain...
6,1.0,Pane-Option-Handler,NaN,"{BufferOptions, PluginManagerOptionPane.Mirror...",Pane-Option-Handler,BufferOptions PluginManagerOptionPane.MirrorMo...
5,1.0,BSH-Class-Bsh,NaN,"{BshIterator, CollectionManager.BasicBshIterat...",BSH-Class-Bsh,BshIterator CollectionManager.BasicBshIterator...
4,1.0,VFS-Task-Browser,NaN,"{TaskMonitor.TaskTableEditor, MirrorList, VFSD...",VFS-Task-Browser,TaskMonitor.TaskTableEditor MirrorList VFSDire...
3,1.0,Widget-Factory-Handler,NaN,"{PropertiesChanged, EditBus.HandlerList, View....",Widget-Factory-Handler,PropertiesChanged EditBus.HandlerList View.Foc...
2,1.0,Handler-Text-Area,False,{ErrorsWidgetFactory.ErrorDialog.MyActionListe...,Handler-Text-Area,ErrorsWidgetFactory.ErrorDialog.MyActionListen...


In [14]:

o_nodes = pd.read_csv("jedit-nodes.csv")
o_edges = pd.read_csv("jedit-edges.csv")

remains = o_nodes[~o_nodes['name'].isin(all)].copy()
remains.drop(columns=['type'], inplace=True)
remains['level'] = [0 for _ in remains['name']]
remains['id'] = remains['name']
remains['members'] = [set([name]) for name in remains['name']]
remains['members_str'] = remains['name']

remains

,id,name,package,level,members,members_str
15,CalculateSize,CalculateSize,de.masters_of_disaster.ant.tasks.calculatesize,0,{CalculateSize},CalculateSize
120,BrowserView.LoadingPlaceholder,BrowserView.LoadingPlaceholder,org.gjt.sp.jedit.browser,0,{BrowserView.LoadingPlaceholder},BrowserView.LoadingPlaceholder
145,VFSBrowser.VFSFileFilterRenderer,VFSBrowser.VFSFileFilterRenderer,org.gjt.sp.jedit.browser,0,{VFSBrowser.VFSFileFilterRenderer},VFSBrowser.VFSFileFilterRenderer
147,VFSDirectoryEntryTable.ClearTypeSelect,VFSDirectoryEntryTable.ClearTypeSelect,org.gjt.sp.jedit.browser,0,{VFSDirectoryEntryTable.ClearTypeSelect},VFSDirectoryEntryTable.ClearTypeSelect
148,VFSDirectoryEntryTable.ColumnDragHook,VFSDirectoryEntryTable.ColumnDragHook,org.gjt.sp.jedit.browser,0,{VFSDirectoryEntryTable.ColumnDragHook},VFSDirectoryEntryTable.ColumnDragHook
156,VFSDirectoryEntryTableModel.ExtendedAttribute,VFSDirectoryEntryTableModel.ExtendedAttribute,org.gjt.sp.jedit.browser,0,{VFSDirectoryEntryTableModel.ExtendedAttribute},VFSDirectoryEntryTableModel.ExtendedAttribute
235,NameSpace.CommandPathEntry,NameSpace.CommandPathEntry,org.gjt.sp.jedit.bsh,0,{NameSpace.CommandPathEntry},NameSpace.CommandPathEntry
248,Remote,Remote,org.gjt.sp.jedit.bsh,0,{Remote},Remote
329,JEditDataFlavor,JEditDataFlavor,org.gjt.sp.jedit.datatransfer,0,{JEditDataFlavor},JEditDataFlavor
359,AnimatedIcon,AnimatedIcon,org.gjt.sp.jedit.gui,0,{AnimatedIcon},AnimatedIcon


In [15]:
c_nodes = c_nodes.append(remains, ignore_index = True)
words = [' '.join(re.findall(r'[A-Z](?:[a-z]+|[A-Z]*(?=[A-Z]|$))', ' '.join(members))) for members in c_nodes['members']]
c_nodes['words'] = words


In [16]:
df = c_nodes

df

,level,id,wordcloud.sync,members,name,members_str,package,words
0,1.0,Window-Dockable-Handler,False,"{DockableWindowManagerImpl.Entry, DockableWind...",Window-Dockable-Handler,DockableWindowManagerImpl.Entry DockableWindow...,NaN,Dockable Window Manager Impl Entry Dockable Wi...
1,1.0,Panel-Handler-Plugin,NaN,"{ManagePanel.KeyboardAction, PluginManager.Act...",Panel-Handler-Plugin,ManagePanel.KeyboardAction PluginManager.Actio...,NaN,Manage Panel Keyboard Action Plugin Manager Ac...
2,1.0,J-Check-Box,False,"{JCheckBoxList.DummyRenderer, JCheckBoxList}",J-Check-Box,JCheckBoxList.DummyRenderer JCheckBoxList,NaN,J Check Box List Dummy Renderer J Check Box List
3,1.0,Check-Box-List,NaN,"{JCheckBoxList.Entry, CheckBoxListModel}",Check-Box-List,JCheckBoxList.Entry CheckBoxListModel,NaN,J Check Box List Entry Check Box List Model
4,1.0,Layout-Extended-Grid,NaN,"{ExtendedGridLayout, ExtendedGridLayoutConstra...",Layout-Extended-Grid,ExtendedGridLayout ExtendedGridLayoutConstrain...,NaN,Extended Grid Layout Extended Grid Layout Cons...
...,...,...,...,...,...,...,...,...
73,0.0,ScrollLayout,NaN,{ScrollLayout},ScrollLayout,ScrollLayout,org.gjt.sp.jedit.textarea,Scroll Layout
74,0.0,TextAreaBorder,NaN,{TextAreaBorder},TextAreaBorder,TextAreaBorder,org.gjt.sp.jedit.textarea,Text Area Border
75,0.0,ThreadAbortMonitor,NaN,{ThreadAbortMonitor},ThreadAbortMonitor,ThreadAbortMonitor,org.gjt.sp.util,Thread Abort Monitor
76,0.0,ThreadUtilities.JEditThreadFactory,NaN,{ThreadUtilities.JEditThreadFactory},ThreadUtilities.JEditThreadFactory,ThreadUtilities.JEditThreadFactory,org.gjt.sp.util,Thread Utilities J Edit Thread Factory


In [17]:
connections = set()

for _, x in c_nodes.iterrows():
  for _, y in c_nodes.iterrows():
    x_nodes = list(o_nodes.loc[o_nodes['name'].isin(x['members'])]['id'])
    y_nodes = list(o_nodes.loc[o_nodes['name'].isin(y['members'])]['id'])
    x2y = list(o_edges.loc[o_edges['source'].isin(x_nodes) & o_edges['target'].isin(y_nodes)]['interaction'])
    for interaction in x2y:
      connections.add(f"{x['name']} {y['name']} {interaction}")
      # {
      #   'source': x['name'],
      #   'target': y['name'],
      #   'interaction': interaction,
      # }

connections = [tuple(row.split(' ')) for row in connections]
connections = [{'source':s, 'target':t, 'interaction':i} for s,t,i in connections]


In [18]:

c_edges = pd.DataFrame(connections)

df = c_edges
pd.concat([df.head(5), df.tail(5)])

,source,target,interaction
0,Printer-Print-Pane,VFS-Task-Browser,depends
1,Pane-Option-Handler,Pane-Option-Handler,holds
2,Panel-Handler-Plugin,Widget-Factory-Handler,specializes
3,Abbrevs-Handler-Abbrev,Handler-Text-Area,depends
4,Widget-Factory-Handler,Window-Dockable-Handler,constructs
291,Option-Options-Dialog,Option-Options-Dialog,returns
292,Widget-Factory-Handler,VFS-Task-Browser,constructs
293,Option-Options-Dialog,Option-Options-Dialog,accepts
294,Handler-Text-Area,Panel-Handler-Plugin,accepts
295,Panel-Handler-Plugin,Widget-Factory-Handler,holds


In [ ]:
import requests

exec(open("p4c_init.py").read())
IPython.display.Javascript(_PY4CYTOSCAPE_BROWSER_CLIENT_JS) # Start browser client

In [ ]:
import py4cytoscape as p4c

p4c.cytoscape_ping()
p4c.cytoscape_version_info()

In [ ]:
p4c.create_network_from_data_frames(c_nodes.drop(columns=['members']), c_edges, title="JEdit-highlevel-louvain", collection="JEdit")

In [ ]:
p4c.set_visual_style('Marquee')
p4c.notebook_export_show_image()